In [25]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import r2_score

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Bus

loading and preprocessing

In [21]:
bus = pd.read_csv('cleaned_bus.csv')
bus.info()

# Convert 'date' and 'time' into datetime features
bus['datetime'] = pd.to_datetime(bus['date'] + ' ' + bus['time'])
bus['day_of_week'] = bus['day']
bus['hour'] = bus['datetime'].dt.hour
bus['month'] = bus['datetime'].dt.month
bus['day'] = bus['datetime'].dt.day

# Encode categorical variables
encoder = LabelEncoder()
bus['location'] = encoder.fit_transform(bus['location'])
bus['incident'] = encoder.fit_transform(bus['incident'])
bus['route'] = encoder.fit_transform(bus['route'])
bus['direction'] = encoder.fit_transform(bus['direction'])
bus['day_of_week'] = encoder.fit_transform(bus['day'])
bus['day_type'] = encoder.fit_transform(bus['day_type'])
bus['rush_type'] = encoder.fit_transform(bus['rush_type'])
bus['season'] = encoder.fit_transform(bus['season'])


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54609 entries, 0 to 54608
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   date             54609 non-null  object 
 1   route            54059 non-null  object 
 2   time             54609 non-null  object 
 3   day              54609 non-null  object 
 4   location         54609 non-null  object 
 5   incident         54609 non-null  object 
 6   min_delay        54609 non-null  int64  
 7   min_gap          54609 non-null  int64  
 8   direction        46007 non-null  object 
 9   day_type         54609 non-null  object 
 10  rush_type        54609 non-null  object 
 11  season           54609 non-null  object 
 12  is_holiday       54609 non-null  int64  
 13  max_temp_c       54609 non-null  float64
 14  min_temp_c       54609 non-null  float64
 15  total_precip_mm  54609 non-null  float64
 16  snow_on_grnd_cm  54609 non-null  int64  
dtypes: float64(3

In [22]:
# Create a binary target for delay most likely occur (severe)
bus['delay_occurrence'] = (bus['min_delay'] > 0).astype(int)

# Select features
features = ['hour', 'day', 'month', 'day_of_week', 'location', 'incident', 'route', 'direction',
            'day_type', 'rush_type', 'season', 'is_holiday', 'max_temp_c', 'min_temp_c',
            'total_precip_mm', 'snow_on_grnd_cm']

## Occurrency

In [13]:
# Split data into train and test sets
X = bus[features]
y = bus['delay_occurrence']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest Classifier
rf_occurrence = RandomForestClassifier(random_state=42)
rf_occurrence.fit(X_train, y_train)

# Predictions
y_pred_occurrence = rf_occurrence.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred_occurrence))
print(classification_report(y_test, y_pred_occurrence))

Accuracy: 0.9222669840688519
              precision    recall  f1-score   support

           0       0.77      0.50      0.61      1309
           1       0.93      0.98      0.96      9613

    accuracy                           0.92     10922
   macro avg       0.85      0.74      0.78     10922
weighted avg       0.92      0.92      0.91     10922



## Length

In [31]:
# Split data into train and test sets
X = bus[features]
y = bus['min_delay']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Instantiate the model
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
rf_regressor.fit(X_train, y_train)

# Predict on test data
y_pred = rf_regressor.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')
print(f'R-squared: {r2}')

Mean Squared Error: 1676.6484673060688
Mean Absolute Error: 11.811802878201272
R-squared: 0.4015687445549774


# Streetcar

In [32]:
# Load the TTC delay dataset
streetcar = pd.read_csv('cleaned_streetcar.csv')

# Convert 'date' and 'time' into datetime features
streetcar['datetime'] = pd.to_datetime(streetcar['date'] + ' ' + streetcar['time'])
streetcar['day_of_week'] = streetcar['day']
streetcar['hour'] = streetcar['datetime'].dt.hour
streetcar['month'] = streetcar['datetime'].dt.month
streetcar['day'] = streetcar['datetime'].dt.day

# Encode categorical variables (e.g., 'location', 'incident', 'route', 'day_type')
encoder = LabelEncoder()
streetcar['location'] = encoder.fit_transform(streetcar['location'])
streetcar['incident'] = encoder.fit_transform(streetcar['incident'])
streetcar['line'] = encoder.fit_transform(streetcar['line'])
streetcar['bound'] = encoder.fit_transform(streetcar['bound'])
streetcar['day_of_week'] = encoder.fit_transform(streetcar['day'])
streetcar['day_type'] = encoder.fit_transform(streetcar['day_type'])
streetcar['rush_type'] = encoder.fit_transform(streetcar['rush_type'])
streetcar['season'] = encoder.fit_transform(streetcar['season'])

# Select features
features = ['hour', 'day', 'month', 'day_of_week', 'location', 'incident', 'line', 'bound',
            'day_type', 'rush_type', 'season', 'is_holiday', 'max_temp_c', 'min_temp_c',
            'total_precip_mm', 'snow_on_grnd_cm']

## Occurrency

In [15]:
# Create a binary target for delay occurrence
streetcar['delay_occurrence'] = (streetcar['min_delay'] > 0).astype(int)

# Split data into train and test sets
X = streetcar[features]
y = streetcar['delay_occurrence']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest Classifier
rf_occurrence = RandomForestClassifier(random_state=42)
rf_occurrence.fit(X_train, y_train)

# Predictions
y_pred_occurrence = rf_occurrence.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred_occurrence))
print(classification_report(y_test, y_pred_occurrence))

Accuracy: 0.8707093821510298
              precision    recall  f1-score   support

           0       0.80      0.40      0.53       484
           1       0.88      0.98      0.92      2138

    accuracy                           0.87      2622
   macro avg       0.84      0.69      0.73      2622
weighted avg       0.86      0.87      0.85      2622



## Length

In [52]:
# Split data into train and test sets
X = streetcar[features]
y = streetcar['min_delay']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Instantiate the model
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
rf_regressor.fit(X_train, y_train)

# Predict on test data
y_pred = rf_regressor.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')
print(f'R-squared: {r2}')

Mean Squared Error: 1236.2823183225041
Mean Absolute Error: 13.226827393059942
R-squared: 0.032810975640353734


# Subway

In [46]:
# Load the TTC subway delay dataset
subway = pd.read_csv('cleaned_subway.csv')

# Convert 'date' and 'time' into datetime features
subway['datetime'] = pd.to_datetime(subway['date'] + ' ' + subway['time'])
subway['hour'] = subway['datetime'].dt.hour
subway['day_of_week'] = subway['datetime'].dt.dayofweek
subway['month'] = subway['datetime'].dt.month
subway['day'] = subway['datetime'].dt.day

# Initialize LabelEncoder
encoder = LabelEncoder()

# Encode categorical variables
subway['station'] = encoder.fit_transform(subway['station'])
subway['code'] = encoder.fit_transform(subway['code'])

# Handle missing values in 'bound' and 'line_code' by filling with a placeholder (e.g., 'Unknown')
subway['bound'].fillna('Unknown', inplace=True)
subway['line_code'].fillna('Unknown', inplace=True)
subway['bound'] = encoder.fit_transform(subway['bound'])
subway['line_code'] = encoder.fit_transform(subway['line_code'])

# Encode other categorical columns
subway['day_type'] = encoder.fit_transform(subway['day_type'])
subway['rush_type'] = encoder.fit_transform(subway['rush_type'])
subway['season'] = encoder.fit_transform(subway['season'])

# Select features for the model
features = [
    'hour', 'day', 'month', 'day_of_week', 'station', 'code', 'bound', 'line_code',
    'day_type', 'rush_type', 'season', 'is_holiday', 'max_temp_c', 'min_temp_c',
    'total_precip_mm', 'snow_on_grnd_cm'
]

# Prepare the feature matrix
X = subway[features].values

# Standardize numerical features (optional, if using a model sensitive to feature scaling)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

<ipython-input-46-0d26d5dbca9b>:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  subway['bound'].fillna('Unknown', inplace=True)
<ipython-input-46-0d26d5dbca9b>:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usi

## Occurrency

In [47]:
# Create a binary target for delay occurrence
subway['delay_occurrence'] = (subway['min_delay'] > 0).astype(int)

# Split data into train and test sets
X = subway[features]
y = subway['delay_occurrence']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest Classifier
rf_occurrence = RandomForestClassifier(random_state=42)
rf_occurrence.fit(X_train, y_train)

# Predictions
y_pred_occurrence = rf_occurrence.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred_occurrence))
print(classification_report(y_test, y_pred_occurrence))

Accuracy: 0.7735890652557319
              precision    recall  f1-score   support

           0       0.83      0.81      0.82      2889
           1       0.68      0.72      0.70      1647

    accuracy                           0.77      4536
   macro avg       0.76      0.76      0.76      4536
weighted avg       0.78      0.77      0.77      4536



## Length

In [42]:
features = [
    'hour', 'day', 'month', 'day_of_week', 'station', 'code', 'bound',
    'day_type', 'rush_type', 'season', 'is_holiday']

# Split data into train and test sets
X = subway[features]
y = subway['min_delay']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Instantiate the model
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
rf_regressor.fit(X_train, y_train)

# Predict on test data
y_pred = rf_regressor.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')
print(f'R-squared: {r2}')

Mean Squared Error: 48.93389357331211
Mean Absolute Error: 2.6807790349164353
R-squared: 0.1014800466185879
